<a href="https://colab.research.google.com/github/eshmaapps/kaggleUbiComp/blob/main/mar_ubi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [14]:
!kaggle kernels output robikscube/fast-data-loading-and-low-mem-with-parquet-files/data?select=train_low_mem.parquet -p .

Kernel log downloaded to ./fast-data-loading-and-low-mem-with-parquet-files.log 


In [10]:

!echo '{"username":"hackerinheels","key":"82bbcfeae33614a0dabfa494a6ec3573"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [16]:
!kaggle datasets download -d robikscube/ubiquant-parquet -f train_low_mem.parquet

100% 3.06G/3.07G [00:34<00:00, 70.0MB/s]
100% 3.07G/3.07G [00:34<00:00, 95.5MB/s]


In [4]:
import pyarrow
import pyarrow.parquet

##Load data
import pandas as pd
import numpy as np


In [5]:

# Load the data
#df = pd.read_parquet('train.parquet')
df = pd.read_parquet('drive/MyDrive/train_low_mem.parquet')
# Lets look at the data
df.head()

,row_id,time_id,investment_id,target,f_0,f_1,f_2,f_3,f_4,f_5,...,f_290,f_291,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299
0,0_1,0,1,-0.300875,0.932573,0.113691,-0.402206,0.378386,-0.203938,-0.413469,...,0.366028,-1.095620,0.200075,0.819155,0.941183,-0.086764,-1.087009,-1.044826,-0.287605,0.321566
1,0_2,0,2,-0.231040,0.810802,-0.514115,0.742368,-0.616673,-0.194255,1.771210,...,-0.154193,0.912726,-0.734579,0.819155,0.941183,-0.387617,-1.087009,-0.929529,-0.974060,-0.343624
2,0_6,0,6,0.568807,0.393974,0.615937,0.567806,-0.607963,0.068883,-1.083155,...,-0.138020,0.912726,-0.551904,-1.220772,-1.060166,-0.219097,-1.087009,-0.612428,-0.113944,0.243608
3,0_7,0,7,-1.064780,-2.343535,-0.011870,1.874606,-0.606346,-0.586827,-0.815737,...,0.382201,0.912726,-0.266359,-1.220772,0.941183,-0.609113,0.104928,-0.783423,1.151730,-0.773309
4,0_8,0,8,-0.531940,0.842057,-0.262993,2.330030,-0.583422,-0.618392,-0.742814,...,-0.170365,0.912726,-0.741355,-1.220772,0.941183,-0.588445,0.104928,0.753279,1.345611,-0.737624


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(16,5))
plt.subplot(1,2,1)
sns.distplot(df['f_0'])
plt.subplot(1,2,2)
sns.distplot(df['f_0'])
plt.show()

sns.boxplot(df['placement_exam_marks'])


In [6]:
y = df.target
x = df
x = x.drop(['target', 'row_id'], axis=1)

In [7]:
import pickle
filename = 'mar_model.sav'



In [8]:
!pip install pytorch-tabnet
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error

regressor = TabNetRegressor(verbose=0,seed=42)


In [15]:
import gc
gc.collect()

88

In [ ]:
tscv = TimeSeriesSplit(n_splits = 20)
rmse = []
CV_score_array    =[]
predictions_array =[]

for train_index, test_index in tscv.split(x):
    X_train, X_valid = x.iloc[train_index], x.iloc[test_index]
    y_train, y_valid = y[train_index], y[test_index]

    X_train      = X_train.to_numpy()
    y_train = y_train.to_numpy().reshape(-1, 1)
    
    X_valid      = X_valid.to_numpy()
    y_valid = y_valid.to_numpy().reshape(-1, 1)
    
    regressor.fit(X_train=X_train, y_train=y_train,
              eval_set=[(X_valid, y_valid)],
              patience=300, max_epochs=100,
              eval_metric=['rmse'])
    predictions = regressor.predict(X_valid)
    # calculate Pearson's correlation
    #corr, _ = pearsonr(y_valid, predictions)
    #print('Pearsons correlation: %.3f' % corr)
    pickle.dump(regressor, open(filename, 'wb'))

    CV_score_array.append(regressor.best_cost)
    print("The CV score is %.5f" % np.mean(CV_score_array,axis=0) )
    predictions_array.append(np.expm1(predictions))

Stop training because you reached max_epochs = 100 with best_epoch = 13 and best_val_0_rmse = 0.9372299909591675
Best weights from best epoch are automatically used!
The CV score is 0.93723
Stop training because you reached max_epochs = 100 with best_epoch = 1 and best_val_0_rmse = 0.9178900122642517
Best weights from best epoch are automatically used!
The CV score is 0.92756
Stop training because you reached max_epochs = 100 with best_epoch = 7 and best_val_0_rmse = 0.9214500188827515
Best weights from best epoch are automatically used!
The CV score is 0.92552
Stop training because you reached max_epochs = 100 with best_epoch = 2 and best_val_0_rmse = 0.8964999914169312
Best weights from best epoch are automatically used!
The CV score is 0.91827
Stop training because you reached max_epochs = 100 with best_epoch = 4 and best_val_0_rmse = 0.8823800086975098
Best weights from best epoch are automatically used!
The CV score is 0.91109


In [ ]:
!cp filename "/content/drive/My Drive/"


In [ ]:
# load the model from disk
regressor = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)
#print(result)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
